<a href="https://colab.research.google.com/github/andres-stack/MarchMadnessMachineLearning/blob/main/MarchMadness23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [518]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import preprocessing
import numpy as np
from sklearn import metrics

In [ ]:
location = "/content/drive/MyDrive/MarchMadnessData/march-machine-learning-mania-2023/"
data = pd.read_csv(location + "MRegularSeasonDetailedResults.csv")
data.columns.values

In [ ]:
winners = pd.DataFrame()
losers = pd.DataFrame()
c = ['Season', 'TeamID', 'totalPoints', 'OpposingPoints'
      , 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OpposingFGM', 'OpposingFGA',
       'OpposingFGM3', 'OpposingFGA3', 'OpposingFTM', 'OpposingFTA', 'OpposingOR', 'OpposingDR', 'OpposingAst', 'OpposingTO',
       'OpposingStl', 'OpposingBlk', 'OpposingPF']

winners[c] = data[['Season', 'WTeamID', 'WScore', 'LScore',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF']]
winners['wins'] = 1
winners['losses'] = 0

losers[c] = data[['Season', 'LTeamID', 'LScore', 'WScore',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF']]


losers['wins'] = 0
losers['losses'] = 1

allTeams = pd.concat([winners, losers])
display(allTeams)

In [ ]:
singleTeamSeasons = allTeams.groupby(['Season', 'TeamID']).sum()
singleTeamSeasons["totalGamesPlayed"] = singleTeamSeasons['wins'] + singleTeamSeasons["losses"]
singleTeamSeasons.columns.values

In [522]:
#calculating useful season stats for each team
regSeasonStats = pd.DataFrame()
regSeasonStats["blksPerGame"] = singleTeamSeasons["Blk"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["StlsPerGame"] = singleTeamSeasons["Stl"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["PFPerGame"] = singleTeamSeasons["PF"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["TurnOversPerGames"] = singleTeamSeasons["TO"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["AstPerGame"] = singleTeamSeasons["Ast"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["DefensiveReboundsPerGame"] = singleTeamSeasons["DR"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["OffensiveReboundsPerGame"] = singleTeamSeasons["OR"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["PointsPerGame"] = singleTeamSeasons["totalPoints"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["WinRatio"] = singleTeamSeasons["wins"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["PointRation"] = singleTeamSeasons["totalPoints"]/singleTeamSeasons["OpposingPoints"]
regSeasonStats["OTPerGame"] = singleTeamSeasons["NumOT"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["FG%"] = singleTeamSeasons["FGM"]/singleTeamSeasons["FGA"]
regSeasonStats["FGPerGame"] = singleTeamSeasons["FGM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["3FG%"] = singleTeamSeasons["FGM3"]/singleTeamSeasons["FGA3"]
regSeasonStats["FG3PerGame"] = singleTeamSeasons["FGM3"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["FreeThrowsMadePerGame"] = singleTeamSeasons["OpposingFTM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["FreeThrow%"] = singleTeamSeasons["FTA"]/singleTeamSeasons["FTM"]
regSeasonStats["FreeThrowAllowed"] = singleTeamSeasons["OpposingFTM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["eFG%"] = (singleTeamSeasons["FGA"] + (.5*(singleTeamSeasons["FGM3"]))/singleTeamSeasons["FGA"])
regSeasonStats["FreeThrowRatio"] = singleTeamSeasons["FTA"]/singleTeamSeasons["FGA"]
regSeasonStats['TOV%'] = singleTeamSeasons["TO"]/(regSeasonStats["FG%"])+((.44 + regSeasonStats["FreeThrow%"])+singleTeamSeasons["TO"])
regSeasonStats['OR%'] = singleTeamSeasons["OR"]/singleTeamSeasons["OR"] +singleTeamSeasons["DR"]
#Sully’s Team Four Factor Rating = ((0.50*eFG%)-(0.30*TOV)+(0.15*OREB)+(0.05*FTR))
regSeasonStats["4FactorRating"] = (((.5)*(regSeasonStats["eFG%"])) - (.3*(regSeasonStats['TOV%']))+(.15*regSeasonStats['OR%'])+ (.05*(regSeasonStats["FreeThrowRatio"])))

#defense wins games
regSeasonStats["FG%Allowed"] = singleTeamSeasons["OpposingFGM"]/singleTeamSeasons["OpposingFGA"]
regSeasonStats["FGAllowedPerGame"] = singleTeamSeasons["OpposingFGM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["3FG%Allowed"] = singleTeamSeasons["OpposingFGM3"]/singleTeamSeasons["OpposingFGA3"]
regSeasonStats["3FGAllowedPerGame"] = singleTeamSeasons["OpposingFGM3"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["TurnOverCausedPerGame"] = singleTeamSeasons["OpposingTO"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["TOAssistDifferential"] = regSeasonStats["AstPerGame"]/regSeasonStats["TurnOverCausedPerGame"]
regSeasonStats["DRRatio"] = singleTeamSeasons["DR"]/singleTeamSeasons["OR"] +singleTeamSeasons["DR"]


In [ ]:
#combining the tournament historical data into one DF
compactData = pd.read_csv(location + "MNCAATourneyCompactResults.csv")
MMWinners = pd.DataFrame()
MMLosers = pd.DataFrame()

MMWinners[["Season", "Team1", "Team2"]] = compactData[["Season", "WTeamID", "LTeamID"]]
MMLosers[["Season", "Team1", "Team2"]] = compactData[["Season", "LTeamID", "WTeamID"]]
MMWinners["outcome"] = 1
MMLosers["outcome"] = 0

WinnersLosers = pd.concat([MMWinners,MMLosers])
WinnersLosers = WinnersLosers[WinnersLosers["Season"] >= 2003].reset_index(drop = True)

display(WinnersLosers)

In [ ]:
#adding seed to WinnersLosers DataFrame
seeding = pd.read_csv(location + "MNCAATourneySeeds.csv")
ConsisceSeeding = seeding.set_index(['Season', 'TeamID'])
ConsisceSeeding.head(10)

Team1Seeds = []
Team2Seeds = []

#adding all seeds in DF to 2 lists
for x in range(len(WinnersLosers)):
  Team1Seeds.append(ConsisceSeeding.loc[(WinnersLosers["Season"][x], WinnersLosers["Team1"][x])].values[0])
for x in range(len(WinnersLosers)):
  Team2Seeds.append(ConsisceSeeding.loc[(WinnersLosers["Season"][x], WinnersLosers["Team2"][x])].values[0])

#removing letters
for x in range(len(Team1Seeds)):
  if len(Team1Seeds[x]) == 4:
    Team1Seeds[x] = int(Team1Seeds[x][1:-1])
  else:
    Team1Seeds[x] = int(Team1Seeds[x][1:])
for x in range(len(Team2Seeds)):
  if len(Team2Seeds[x]) == 4:
    Team2Seeds[x] = int(Team2Seeds[x][1:-1])
  else:
    Team2Seeds[x] = int(Team2Seeds[x][1:])

print(Team1Seeds, Team2Seeds)

#add seeds to DF
WinnersLosers["Team1Seed"] = Team1Seeds
WinnersLosers["Team2Seed"] = Team2Seeds

display(WinnersLosers)

In [ ]:
final = []

for i in range(len(WinnersLosers)):
  index = (WinnersLosers['Season'][i], WinnersLosers['Team1'][i])
  team1Final = regSeasonStats.loc[index]
  team1Final['Seed'] = WinnersLosers['Team1Seed'][i]

  index = (WinnersLosers['Season'][i], WinnersLosers['Team2'][i])
  team2Final = regSeasonStats.loc[index]
  team2Final['Seed'] = WinnersLosers['Team2Seed'][i]

  f = team1Final - team2Final
  f['outcome'] = WinnersLosers['outcome'][i]
  final.append(f)
final = pd.DataFrame(final)

display(final)

In [ ]:
corrs = round(final.corr(),2)
display(np.abs(corrs['outcome']))

In [ ]:
x = final[final.columns[:-1]].values
y = final['outcome'].values

np.random.seed(1)
idx = np.random.permutation(len(x))
train_idx = idx[:int(-.2*len(x))]
test_idx = idx[int(-.2*len(x)):]

X_train = x[train_idx]
X_test = x[test_idx]
Y_train = y[train_idx]
Y_test = y[test_idx]

min = X_train.min(axis=0)
max = X_train.max(axis=0)

X_train = (X_train - min)/(max - min)
X_test = (X_test - min)/(max - min)

poly = PolynomialFeatures(degree = 4, interaction_only=False, include_bias=False)
X_poly = poly.fit_transform(X_train)
#training utilizing logistic regression model
logreg = LogisticRegression(random_state=16)
logreg.fit(X_poly, Y_train)
logreg.score(poly.transform(X_test), Y_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)
scaler = preprocessing.StandardScaler().fit(X_train)
scaler = preprocessing.StandardScaler().fit(X_test)
X_testScaled = scaler.transform(X_test)
X_scaled = scaler.transform(X_train)
svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_scaled, y_train)
y_pred = svclassifier.predict(X_testScaled)
print("Gaussian Kernel Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
#Support Vector Machine with polynomial kernal 
polyclassifier = SVC(kernel = 'poly', degree = 5, probability=True)
polyclassifier.fit(X_scaled, y_train)
prediction = polyclassifier.predict(X_testScaled)
print('polynomial degree 5'':',metrics.accuracy_score(y_test, prediction))

In [ ]:
print(type(X_testScaled))

In [ ]:
data = pd.read_csv(location + "newMRegularSeasonDetailedResults (1).csv")
matchups = pd.read_csv(location + "newMNCAATourneySeeds.csv")
teams = pd.read_csv(location + "MTeams.csv")
data = data[data["Season"] >= 2023].reset_index(drop = True)

winners = pd.DataFrame()
losers = pd.DataFrame()
c = ['Season', 'TeamID', 'totalPoints', 'OpposingPoints'
      , 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA',
       'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OpposingFGM', 'OpposingFGA',
       'OpposingFGM3', 'OpposingFGA3', 'OpposingFTM', 'OpposingFTA', 'OpposingOR', 'OpposingDR', 'OpposingAst', 'OpposingTO',
       'OpposingStl', 'OpposingBlk', 'OpposingPF']

winners[c] = data[['Season', 'WTeamID', 'WScore', 'LScore',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA',
       'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA',
       'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO',
       'LStl', 'LBlk', 'LPF']]
winners['wins'] = 1
winners['losses'] = 0

losers[c] = data[['Season', 'LTeamID', 'LScore', 'WScore',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA',
       'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA',
       'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO',
       'WStl', 'WBlk', 'WPF']]


losers['wins'] = 0
losers['losses'] = 1

allTeams = pd.concat([winners, losers])
display(allTeams)

In [ ]:
singleTeamSeasons = allTeams.groupby(['TeamID']).sum()
singleTeamSeasons["totalGamesPlayed"] = singleTeamSeasons['wins'] + singleTeamSeasons["losses"]
singleTeamSeasons.reset_index(inplace=True)
singleTeamSeasons.columns.values

In [ ]:
#calculating useful season stats for each team
regSeasonStats = pd.DataFrame()
regSeasonStats['TeamID'] = singleTeamSeasons["TeamID"]
regSeasonStats["blksPerGame"] = singleTeamSeasons["Blk"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["StlsPerGame"] = singleTeamSeasons["Stl"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["PFPerGame"] = singleTeamSeasons["PF"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["TurnOversPerGames"] = singleTeamSeasons["TO"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["AstPerGame"] = singleTeamSeasons["Ast"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["DefensiveReboundsPerGame"] = singleTeamSeasons["DR"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["OffensiveReboundsPerGame"] = singleTeamSeasons["OR"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["PointsPerGame"] = singleTeamSeasons["totalPoints"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["WinRatio"] = singleTeamSeasons["wins"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["PointRation"] = singleTeamSeasons["totalPoints"]/singleTeamSeasons["OpposingPoints"]
regSeasonStats["OTPerGame"] = singleTeamSeasons["NumOT"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["FG%"] = singleTeamSeasons["FGM"]/singleTeamSeasons["FGA"]
regSeasonStats["FGPerGame"] = singleTeamSeasons["FGM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["3FG%"] = singleTeamSeasons["FGM3"]/singleTeamSeasons["FGA3"]
regSeasonStats["FG3PerGame"] = singleTeamSeasons["FGM3"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["FreeThrowsMadePerGame"] = singleTeamSeasons["OpposingFTM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["FreeThrow%"] = singleTeamSeasons["FTA"]/singleTeamSeasons["FTM"]
regSeasonStats["FreeThrowAllowed"] = singleTeamSeasons["OpposingFTM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["eFG%"] = (singleTeamSeasons["FGA"] + (.5*(singleTeamSeasons["FGM3"]))/singleTeamSeasons["FGA"])
regSeasonStats["FreeThrowRatio"] = singleTeamSeasons["FTA"]/singleTeamSeasons["FGA"]
regSeasonStats['TOV%'] = singleTeamSeasons["TO"]/(regSeasonStats["FG%"])+((.44 + regSeasonStats["FreeThrow%"])+singleTeamSeasons["TO"])
regSeasonStats['OR%'] = singleTeamSeasons["OR"]/singleTeamSeasons["OR"] +singleTeamSeasons["DR"]
#Sully’s Team Four Factor Rating = ((0.50*eFG%)-(0.30*TOV)+(0.15*OREB)+(0.05*FTR))
regSeasonStats["4FactorRating"] = (((.5)*(regSeasonStats["eFG%"])) - (.3*(regSeasonStats['TOV%']))+(.15*regSeasonStats['OR%'])+ (.05*(regSeasonStats["FreeThrowRatio"])))

#defense wins games
regSeasonStats["FG%Allowed"] = singleTeamSeasons["OpposingFGM"]/singleTeamSeasons["OpposingFGA"]
regSeasonStats["FGAllowedPerGame"] = singleTeamSeasons["OpposingFGM"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["3FG%Allowed"] = singleTeamSeasons["OpposingFGM3"]/singleTeamSeasons["OpposingFGA3"]
regSeasonStats["3FGAllowedPerGame"] = singleTeamSeasons["OpposingFGM3"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["TurnOverCausedPerGame"] = singleTeamSeasons["OpposingTO"]/singleTeamSeasons["totalGamesPlayed"]
regSeasonStats["TOAssistDifferential"] = regSeasonStats["AstPerGame"]/regSeasonStats["TurnOverCausedPerGame"]
regSeasonStats["DRRatio"] = singleTeamSeasons["DR"]/singleTeamSeasons["OR"] +singleTeamSeasons["DR"]

display(regSeasonStats)

In [ ]:
#adding seed to regSeasonStats DataFrame
seeding = pd.read_csv(location + "MNCAATourneySeeds.csv")
ConsisceSeeding = seeding.set_index(['TeamID'])
ConsisceSeeding = ConsisceSeeding[ConsisceSeeding["Season"] >= 2023].reset_index()
ConsisceSeeding = ConsisceSeeding.drop("Season",axis=1)
ConsisceSeeding = ConsisceSeeding.set_index("TeamID")

#add seed
TeamSeeds = []
for x in range(len(regSeasonStats)):
  try:
    TeamSeeds.append(ConsisceSeeding.loc[regSeasonStats["TeamID"][x]]["Seed"])
  except:
    TeamSeeds.append('w01')
  
for x in range(len(TeamSeeds)):
  if len(TeamSeeds[x]) == 4:
    TeamSeeds[x] = int(TeamSeeds[x][1:-1])
  else:
    TeamSeeds[x] = int(TeamSeeds[x][1:])

#add seeds to DF
regSeasonStats["Seed"] = TeamSeeds

#remove index and TeamID
#convert row into dataframe and subtract them
regSeasonStats.loc[:,"TeamID"]
df_new = regSeasonStats[regSeasonStats['TeamID'] == 1222] #ver
df_new2 = regSeasonStats[regSeasonStats['TeamID'] == 1345] #nk
df_new.reset_index(inplace=True)
df_new2.reset_index(inplace=True)
df_newerererer = (df_new2-df_new)
df_newerererer = df_newerererer.drop("index", axis = 1)
df_newerererer = df_newerererer.drop("TeamID", axis = 1)
df_newerererer = scaler.transform(df_newerererer)
svclassifier.predict(df_newerererer)

# 1 means team 2 wins